In [1]:
from dataclasses import dataclass

from autogen_core import (
    MessageContext,
    RoutedAgent,
    SingleThreadedAgentRuntime,
    TopicId,
    TypeSubscription,
    message_handler,
    type_subscription,
)
from IPython.display import display, Markdown

In [2]:
# Message protocol
@dataclass
class Message:
    content: str

# Topic
concept_extractor_topic_type = "ConceptExtractorAgent"
writer_topic_type = "WriterAgent"
format_proof_topic_type = "FormatProofAgent"
user_topic_type = "UserAgent"

# Concept Extractor Agent   

In [3]:
from autogen_core.models import ChatCompletionClient, SystemMessage, UserMessage, AssistantMessage
@type_subscription(topic_type=concept_extractor_topic_type)
class ConceptExtractorAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient):
        super().__init__("A concept extractor agent")
        self._model_client = model_client
        self._system_message = SystemMessage(
            content=(
                "You are a marketing analyst. Given a product description, identify:\n"
                "- Key features\n"
                "- Target audience\n"
                "- Unique selling points\n\n"
            )
        )

    @message_handler
    async def handle_user_description(
        self, message: Message, context: MessageContext
    ) -> None:
        user_message = UserMessage(content=message.content, source="user")
        response = await self._model_client.create(
            messages=[self._system_message, user_message],
            cancellation_token=context.cancellation_token,
        )
        extracted_concepts = response.content
        assert isinstance(extracted_concepts, str), "Response should be a string"
        display(Markdown(f"**{self.id.key}:**\n{extracted_concepts}"))

        await self.publish_message(message=Message(content=extracted_concepts), topic_id=TopicId(writer_topic_type, self.id.key))

# Writer Agent!

In [4]:
@type_subscription(topic_type=writer_topic_type)
class WriterAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient):
        super().__init__("A writer agent")
        self._model_client = model_client
        self._system_message = SystemMessage(
            content=(
                "You are a marketing copywriter. Given a block of text describing features, audience, and USPs, "
                "compose a compelling marketing copy (like a newsletter section) that highlights these points. "
                "Output should be short (around 150 words), output just the copy as a single text block."
            )
        )

    @message_handler
    async def handle_intermediate_text(
        self, message: Message, context: MessageContext
    ) -> None:
        assistant_message = AssistantMessage(content=message.content, source="assistant")
        response = await self._model_client.create(
            messages=[self._system_message, assistant_message],
            cancellation_token=context.cancellation_token,
        )
        product_description = response.content
        assert isinstance(product_description, str), "Response should be a string"
        display(Markdown(f"**{self.id.key}:**\n{product_description}"))

        await self.publish_message(message=Message(content=product_description), topic_id=TopicId(format_proof_topic_type, self.id.key))

# Format Proof Agent

In [5]:
@type_subscription(topic_type=writer_topic_type)
class FormatProofAgent(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient):
        super().__init__("A format proof agent")
        self._model_client = model_client
        self._system_message = SystemMessage(
            content=(
                "You are an editor. Given the draft copy, correct grammar, improve clarity, ensure consistent tone, "
                "give format and make it polished. Output the final improved copy as a single text block."
            )
        )

    @message_handler
    async def handle_intermediate_text(
        self, message: Message, context: MessageContext
    ) -> None:
        assistant_message = AssistantMessage(content=f"Draft copy: {message.content}" , source="user")
        response = await self._model_client.create(
            messages=[self._system_message, assistant_message],
            cancellation_token=context.cancellation_token,
        )
        formatted_text = response.content
        assert isinstance(formatted_text, str), "Response should be a string"
        display(Markdown(f"**{self.id.key}:**\n{formatted_text}"))

        await self.publish_message(message=Message(content=formatted_text), topic_id=TopicId(user_topic_type, self.id.key))

# User Agent

In [6]:
@type_subscription(topic_type=user_topic_type)
class UserAgent(RoutedAgent):
    def __init__(self):
        super().__init__("A user agent that outputs the final copy to the user.")


    @message_handler
    async def handle_final_copy(
        self, message: Message, context: MessageContext
    ) -> None:
        assistant_message = AssistantMessage(content=message.content, source="assistant")
        display(Markdown(f"**{self.id.key}:**\n{assistant_message.content}"))

# Register to runtime

In [8]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()
model_client = OpenAIChatCompletionClient(
    model="gpt-4o")

runtime = SingleThreadedAgentRuntime()

await ConceptExtractorAgent.register(
    runtime=runtime,
    type=concept_extractor_topic_type,
    factory=lambda: ConceptExtractorAgent(model_client=model_client)
)
await WriterAgent.register(
    runtime=runtime,
    type=writer_topic_type,
    factory=lambda: WriterAgent(model_client=model_client)
)
await FormatProofAgent.register(
    runtime=runtime,
    type=format_proof_topic_type,
    factory=lambda: FormatProofAgent(model_client=model_client)
)
await UserAgent.register(
    runtime=runtime,
    type=user_topic_type,
    factory=lambda: UserAgent()
)

AgentType(type='UserAgent')

# Run workflow

In [9]:
runtime.start()
await runtime.publish_message(
    Message(content="An eco-friendly stainless steel water bottle that keeps drinks cold for 24 hours"),
    topic_id=TopicId(concept_extractor_topic_type, "user")
)
await runtime.stop_when_idle()
await model_client.close()

**user:**
- **Key Features:**
  - Made from stainless steel
  - Eco-friendly design
  - Insulation technology to keep drinks cold for 24 hours

- **Target Audience:**
  - Environmentally conscious consumers
  - Active individuals who need long-lasting cold hydration
  - Outdoor enthusiasts
  - Office workers looking for sustainable products

- **Unique Selling Points:**
  - Extended 24-hour cold retention compared to typical water bottles
  - Durability and reusability due to stainless steel construction
  - Environmentally friendly alternative to single-use plastic bottles

**user:**
**Key Features:**
- Constructed from resilient stainless steel
- Eco-friendly design prioritizing sustainability
- Innovative insulation technology keeps beverages cold for up to 24 hours

**Target Audience:**
- Environmentally conscious consumers
- Active individuals requiring prolonged cold hydration
- Outdoor enthusiasts
- Office workers seeking sustainable product options

**Unique Selling Points:**
- Superior 24-hour cold retention compared to standard water bottles
- Enhanced durability and reusability due to stainless steel construction
- Eco-friendly alternative to single-use plastic bottles

**user:**
**Key Features:**
- Constructed from resilient stainless steel
- Eco-friendly design prioritizing sustainability
- Innovative insulation technology keeps beverages cold for up to 24 hours

**Target Audience:**
- Environmentally conscious consumers
- Active individuals requiring prolonged cold hydration
- Outdoor enthusiasts
- Office workers seeking sustainable product options

**Unique Selling Points:**
- Superior 24-hour cold retention compared to standard water bottles
- Enhanced durability and reusability due to stainless steel construction
- Eco-friendly alternative to single-use plastic bottles

**user:**
Stay refreshed and support the planet with our state-of-the-art, eco-conscious water bottle! Crafted from durable stainless steel, this bottle isn't just about style—it's about sustainability. Engineered with advanced insulation technology, it keeps your beverages ice-cold for a remarkable 24 hours, outlasting typical water bottles. Whether you're hitting the trails, gearing up for a long workday, or simply advocating for a greener planet, our bottle caters to your every need. It's the ideal choice for environmentally aware individuals and active adventurers who demand more from their hydration solutions. Ditch single-use plastics and embrace a bottle that delivers endurance, quality, and eco-friendliness. Make a lasting impact on your health and the environment—choose a hydration partner that cares as much as you do. Grab yours today and transform the way you hydrate!